In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('china_threat.csv',lineterminator='\n')
df

,Unnamed: 0,string_field_2,cnt
0,0,Tampa News Market crash has reached the panic ...,256
1,1,Can people stop panic buying_,256
2,2,I just want people to stop panic buying as the...,256
3,3,Coronavirus Macro Analysis The effect so far h...,256
4,4,Hard to know. The Federal Reserve keeps loweri...,256
...,...,...,...
258542,258542,I think i am gonna have a panic attack,3071
258543,258543,Im having panic attacks for no reason,3583
258544,258544,Local financial experts say although this down...,3583
258545,258545,Im about to have a panic attack all of a sudden,3583


In [ ]:
import jieba
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')


# 格式转换 否则会报错  'float' object has no attribute 'decode'
# df = pd.DataFrame(df['content'].astype(str))
df = pd.DataFrame(df['string_field_2'].astype(str))

def chinese_word_cut(mytext):
    wordless = jieba.lcut(mytext)
    wordless = [w for w in wordless if w not in stopwords and len(w) > 1]
    document = " ".join(wordless)
    return document


# 增加一列数据
# df['content_cutted'] = df['content'].apply(chinese_word_cut)
df['content_cutted'] = df['string_field_2'].apply(chinese_word_cut)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.808 seconds.
Prefix dict has been built successfully.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# 设置特征数
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                max_df=0.5,
                                min_df=10)  # 去除文档内出现几率过大或过小的词汇
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [5]:
from sklearn.decomposition import LatentDirichletAllocation

# 设置主题数
n_topics = 15

lda = LatentDirichletAllocation(n_components=n_topics,
                                max_iter=100,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0)
lda.fit(tf)

# 计算困惑度
print(u'困惑度：')
print(lda.perplexity(tf, sub_sampling=False))
print("")


# 主题-关键词分布
def print_top_words(model, tf_feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):  # lda.component相当于model.topic_word_
        print('Topic #%d:' % topic_idx)
        print(' '.join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print("")


# 定义好函数之后 暂定每个主题输出前30个关键词
n_top_words = 30
tf_feature_names = tf_vectorizer.get_feature_names()
# 调用函数
print_top_words(lda, tf_feature_names, n_top_words)

困惑度：
502.0449341266332

Topic #0:
tariffs chinese billion trump threatened us goods imports impose president donald percent 200 worth products beijing administration trade the tariff threat 25 additional 10 50 threatening another slap would white

Topic #1:
trade us two war threats tariffs threatening threatened steel currency trump european campaign agreement economies property billions largest dollars said trumps worlds countries intellectual months recent the could american exports

Topic #2:
west china8217s many amp threatens 2019 come world main face sign things economic jan forced 14 russia money partnership gold rights zero adds navigation future powerhouse must amthe overtake tonnes

Topic #3:
south military sea chinas us taiwan threat region chinese threatened threatening threaten called stability islands change said beijing actions sovereignty claims island part east regional threats peace navy climate philippines

Topic #4:
north korea nuclear threat us south missile korean 

In [6]:
import pyLDAvis.sklearn

data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
# print(data)
# 显示图形
pyLDAvis.display(data)


/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
